## 0) Chiama i pacchetti e gli script necessari (step preliminare)

In [ ]:
# Import needed packages
import numpy as np
import pandas as pd
#import datetime
import csv
import plotly.graph_objects as go

from SCRIPTS.Yearly_Calc import Yearly_Calculations as  Yearly_Calculations
from SCRIPTS.detailed_economic import Detailed_Economic_Analysis as  Detailed_Economic_Analysis
from SCRIPTS.utils import figure_scatter, figure_bar

# 1) Imposta gli input dell'analisi 
### I campi sottostanti sono gli unici campi da compilare se si vuole far girare il modello senza modificare niente
### Nota Bene: Vanno caricati i file .CSV nella cartella INPUT con i dati orari di producibilità fotovoltaica e i dati orari dei consumi
### Attenzione: usare i template csv forniti per caricare i dati (mantenere i nomi dei file e il nome delle colonne nel template CSV)

### 1 - 0) Scrivi il percorso del file contenente i dati sui consumi orari di tutti i membri della CER

In [ ]:
filename_Load ='INPUT/Consumo_Carichi.csv'

### 1 - 1) Taglia dell'impianto fotovoltaico (kWp) per cui sono stati inseriti i dati di producibilità oraria 

In [ ]:
filename_FV ='INPUT/Produzione_FV.csv'
PV_kWp_dati = 50           #kWp

### 1 - 2) Taglia dell'impianto fotovoltaico (kWp) da simulare

In [ ]:
PV_kWp = 100                #kWp

### 1 - 3) Dati di progetto della batteria

In [ ]:
Capacity_Battery = 0                              # Capacità, kWh
Maximum_Power = 0                                 # Potenza massima erogabile dalla batteria, kW
SOC_min = 5                                       # Stato di carica minimo, %
eta_BC = 0.99                                     # Efficienza di carica
eta_BD = 0.99                                     # Efficienza di scarica
SOC_0 = 50                                        # Stato di carica iniziale, %

### 1 - 4) Input economici

In [ ]:
# Dati investimento - COSTO DI INVESTIMENTO SPECIFICO
CAPEX_PV_specifico = 1200         # eur/kWp
CAPEX_battery_specifico = 500     # eur/kWh

# Dati costi operativi (espressi in % dell'investimento)
OPEX_Perc = 3.5                    # %
Inverter  = 10                     # %

# Scegli la tipologia di schema
# Opzioni 'AC', 'CER'
Tipo = 'CER'                      #'CER' oppure 'AC'

# Dati incentivo (Lasciare invariato se si vuole rispettare la normativa attuale)
Incentivo_MISE     = 0.11  *(Tipo=='CER')  + 0.1*(Tipo=='AC')    # eur/kWh
Sgravio_ARERA      = 0.008 *(Tipo=='CER')  + 0.01*(Tipo=='AC')   # eur/kWh

# Prezzo di mercato dell'energia elettrica
Prezzo_El_Mercato  = 0.05                                        # eur/kWh

# Tasso di attualizzazione - necessario per il calcolo del Valore Attuale Netto
Tasso_Attualizzazione = 5            # % 


# 2) Simulazione annuale

In [ ]:
%run SCRIPTS/run_model.ipynb

%run SCRIPTS/define_figures.ipynb

# 3) Risultati

## 3.0 - Verifica il bilancio energetico

In [ ]:
energy_balance

## 3.1) Consumi 

In [ ]:
figure_consumption = figure_scatter(dict_plot_consumption,title_consumption, xlabel_consumption, ylabel_consumption)
figure_consumption.show()

figure_consumption2 = figure_bar(dict_plot_consumption2,title_consumption2, xlabel_consumption2, ylabel_consumption2)
figure_consumption2.show()

print('Il carico annuale è di ', round(df_hourly['Consumo carichi, kWh'].sum()/1e3,1), ' MWh')

## 3.2 - Produzione fotovoltaico

In [ ]:
figure_PV1 = figure_scatter(dict_plot_PV1,title_PV1, xlabel_PV1, ylabel_PV1)
figure_PV1.show()

figure_PV2 = figure_bar(dict_plot_PV2,title_PV2, xlabel_PV2, ylabel_PV2)
figure_PV2.show()

print('La produzione annuale di un impianto da ' ,str(PV_kWp) , ' kWp risulta ugsuale a  ', 
      round(df_hourly['Produzione PV, kWh'].sum()/1000,2), ' MWh')
print('Le ore equivalenti sono ', 
      round(df_hourly['Produzione PV, kWh'].sum()/PV_kWp,2))

## 3.3- Flussi energetici orari annuali

In [ ]:
figure_energyflux = figure_scatter(dict_plot_energyflux,title_energyflux, xlabel_energyflux, ylabel_energyflux)
figure_energyflux.show()



figure_monthly_overall = figure_bar(dict_plot_monthly_overall,title_monthly_overall, xlabel_monthly_overall, ylabel_monthly_overall)
figure_monthly_overall.show()



figure_monthly_overall2 = figure_bar(dict_plot_monthly_overall2,title_monthly_overall2, xlabel_monthly_overall2, 
                                     ylabel_monthly_overall2)
figure_monthly_overall2.show()


## 3.4 - Analisi del funzionamento della batteria

In [ ]:
figure_battery = figure_scatter(dict_plot_battery,title_battery, xlabel_battery, ylabel_battery)
figure_battery.show()

try:
    print('In un anno di funzionamento, la batteria subisce ', int(economy_output['Cicli batteria']), ' cicli di carica/scarica')
except:
    print('Nessuna batteria installata')

# 4) Risultati: analisi economica

## 4.0) Investimento - Costi operativi e Ricavi

In [ ]:
print('Investimento fotovoltaico, eur', round(economy_output['CAPEX_PV'],0))
print('Investimento accumulo, eur', round(economy_output['CAPEX_battery'] ,0))
print('Investimento totale, eur', round(economy_output['CAPEX_battery']+economy_output['CAPEX_PV'] ,0))
print('Costi operativi, eur/anno', round(economy_output['OPEX'] ,0))
print('Costi riampiazzo inverter (anno 10), eur', round(economy_output['InverterCost'] ,0))

print()

print('Totale energia condivisa PV, kWh ', round(economy_output['Energia_Condivisa_PV'],0))
print('Totale energia condivisa batteria, kWh ', round(economy_output['Energia_Condivisa_Batteria'],0))
print('Totale energia condivisa, kWh ', round(economy_output['Energia_Condivisa'],0))
print()
print('% energia condivisa ', round(economy_output['Energia_Condivisa']/economy_output['Energia_Consumi']*100), ' %')
print('% energia condivisa Fotovoltaico ', round(economy_output['Energia_Condivisa_PV']/economy_output['Energia_Consumi']*100), ' %')
print('% energia condivisa Batteria ', round(economy_output['Energia_Condivisa_Batteria']/economy_output['Energia_Consumi']*100), ' %')
print()
print('Totale energia consumata, kWh ', round(economy_output['Energia_Consumi'],0))
print('Totale energia immessa in rete (totale), kWh ', round(economy_output['Energia_Immessa_In_Rete'],0))
print('Totale energia prelevata dalla rete (totale), kWh ', round(economy_output['Energia_Prelevata_tot'],0))
print()


print('Incentivo MISE, eur/anno ', round(economy_output['Ritorno_MISE'],0))
print('Sgravio ARERA, eur/anno ', round(economy_output['Ritorno_ARERA'],0))
print('Vendita energia elettrica, eur/anno ', round(economy_output['Ritorno_RID'],0))
print('Totale ricavi, eur/anno ', round(economy_output['Totale_Ricavi'],0))


## 4.1 ) Tempo di ritorno dell'investimento e Valore Attuale Netto a 20 anni
NB. Le due curve mostrano lo scenario in cui sono applicate le detrazioni fiscali sul 50 % dell'investimento (per un max investimento di 96.000 EUR) spalmate sui primi 10 anni, e lo scenario in cui nessuna detrazione è applicata

In [ ]:
figure_VAN = figure_scatter(dict_VAN,title_VAN, xlabel_VAN, ylabel_VAN)
figure_VAN.show()

PBT_1 =  np.round(((economy_output['CAPEX_PV']+economy_output['CAPEX_battery'])/(economy_output['Totale_Ricavi']-economy_output['OPEX'])),1)
PBT_detrazioni = np.round(((economy_output['CAPEX_PV']+economy_output['CAPEX_battery'])/(economy_output['Totale_Ricavi']-economy_output['OPEX'] + min((economy_output['CAPEX_PV']+economy_output['CAPEX_battery']), 96000)/10*0.5)),1)
print()
print('Tempo ritorno dell investimento ', PBT_1 , ' anni')
print('Tempo ritorno dell investimento (detrazioni 50 %)',PBT_detrazioni , ' anni')


## 5) Esporta i risultati in un file CSV

In [ ]:
# Scegli il nome del file in cui salvare i risultati
filename_output = 'OUTPUT/OutputSimulazione.csv'

In [ ]:
OUTPUT_DICTIONARY = {'Taglia FV kWp': PV_kWp, 
                     'Taglia batteria kWh': Capacity_Battery, 
                     'Consumi annuali kWh/anno': economy_output['Energia_Consumi'],
                     'Investimento FV EUR': economy_output['CAPEX_PV'],
                     'Investimento accumulo EUR': economy_output['CAPEX_battery'],
                     'Costi operativi EUR/anno': economy_output['OPEX'],
                     'Costi riampiazzo inverter EUR/anno': economy_output['InverterCost'],
                     'Energia condivisa FV kWh/anno': economy_output['Energia_Condivisa_PV'],
                     'Energia condivisa batteria kWh/anno': economy_output['Energia_Condivisa_Batteria'],
                     'Totale energia immessa in rete kWh/anno': economy_output['Energia_Immessa_In_Rete'],
                     'Totale energia prelevata dalla rete kWh/anno': economy_output['Energia_Prelevata_tot'],
                     'Ricavo MISE EUR/anno':economy_output['Ritorno_MISE'],
                     'Sgravio ARERA EUR/anno':economy_output['Ritorno_ARERA'],
                     'Vendita energia elettrica EUR/anno': economy_output['Ritorno_RID'],
                     'Tempo di ritorno senza detrazioni': PBT_1,
                     'Tempo di ritorno con detrazioni': PBT_detrazioni,
                    }

df_OUTPUT = pd.DataFrame.from_dict(OUTPUT_DICTIONARY,orient='index')
df_OUTPUT.to_csv(filename_output, decimal =',', sep=';')